## Google Geocoding

* Now that everyone has an API Key (Congratulations!), it is time to start using it!

* You can utilize the Google Maps Geocoding API to turn addresses into latitudinal and longitudinal coordinates.

  * This process of converting an address to coordinates is called **geocoding**.

  * Since many APIs only understand locations formatted in terms of latitude/longitude, geocoding will be very valuable in translating addresses into data that APIs - like the Google Places API - can understand.

    * Google's API is not free and if credit card information is provided, Google will charge past a certain usage point. **It is a good idea to avoid pushing your API key to github by using adding the `config.py` to their `.gitignore` file or using environment variables.**

In [2]:
# Dependencies
import requests
import json
import pandas as pd

from pprint import pprint

# Google developer API key
import sys
sys.path.append('../..')
from config import gkey
from config import owm_api


file = pd.read_csv("winedata.csv")

file.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [4]:
file["winery name"] = file["winery"] + " Winery"
file.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,winery name
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,Nicosia Winery
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Quinta dos Avidagos Winery
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Rainstorm Winery
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,St. Julian Winery
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Sweet Cheeks Winery


In [5]:
#For test purposes, just looking at one country
just_brazil = file.loc[file["country"]=="Brazil"]

In [6]:
# Target location

winery_response =[]

# Run API calls for each winery in list
for winery in just_brazil["winery name"]:

    target_location = winery + " " + just_brazil["country"]
    
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_location, gkey)
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
   
    # Add response to our list
    winery_response.append(geo_data)        

In [8]:
# Collect and store latitude and longitude for each winery
#winery_list = []
lat = []
lng = []

i=0

# Loop through wineries and append coordinate lists
for w in winery_response:
    try:
        latitude = winery_response[i]["results"][0]["geometry"]["location"]["lat"]
        longitude = winery_response[i]["results"][0]["geometry"]["location"]["lng"]
        
  #      winery_list.append(w)
        lat.append(latitude)
        lng.append(longitude)
        
        i = i+1
    except:
        lat.append("lat error")
        lng.append("lng error")
    
        i = i+1

In [9]:
# Add coordinates to dataframe
just_brazil["latitude"] = lat
just_brazil["longitude"] = lng

C:\Users\Alyza\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Alyza\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Drop rows without location data
#just_brazil = just_brazil.loc[just_brazil["latitude"] != "na"]

In [ ]:
type(just_brazil[["longitude", "latitude"]])
brazil_coords = just_brazil[["longitude", "latitude"]]
type(brazil_coords.iloc[1,1])
brazil_coords

In [ ]:
import gmaps
import numpy as np


# Configure gmaps with API key
gmaps.configure(api_key=gkey)


# Store 'Lat' and 'Lng' into  locations 
#locations = just_brazil[["latitude", "longitude"]].astype(float)

In [ ]:
# Create winery symbol layer
winery_layer = gmaps.marker_layer(
    locations
)


fig = gmaps.figure()
fig.add_layer(winery_layer)

fig

In [ ]:
from config import weather_api

weather_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?q=" 

In [21]:
for latt in brazil_coords:
    print(type(latt))

<class 'str'>
<class 'str'>


In [37]:
x=0
for latt in brazil_coords:
    try:
        num_lat = brazil_coords["latitude"].astype(float)
        print(type(num_lat))
        x+1
    except:
        print("error")

error
error


In [ ]:
for x in just_brazil:
    print(float(x[1]["latitude"]))

In [ ]:
# Northern hemisphere growing season weather lists
jul = []
aug = []
sep = []

# Southern hemisphere growing season weather lists
jan = []
feb = []
mar = []

# Loop through wineries and append weather lists

for y in just_brazil:
    print(type(lat))
    try:
        if float(y["latitude"]) < 0:
            lat = y["latitude"]
            lng = y["longitude"]
            
            print(lat)
            
            print(f"{base_url}{lat},{lng}&key={weather_api}&date=2011-07-01&enddate=2011-07-31&tp=24&format=json")
            
    except:
        
        print("error")
        
            
        
                

In [ ]:
for lat in just_brazil:
    print(type(lat[1]))

In [ ]:
        # Run a request to endpoint and convert result to json
            jul_weather = requests.get(f"{base_url}{lat},{lng}&key={weather_api}&date=2011-07-01&enddate=2011-07-31&tp=24&format=json").json()
            aug_weather = requests.get(f"{base_url}{lat},{lng}&key={weather_api}&date=2011-08-01&enddate=2011-08-31&tp=24&format=json").json()
            sep_weather = requests.get(f"{base_url}{lat},{lng}&key={weather_api}&date=2011-09-01&enddate=2011-09-30&tp=24&format=json").json()
                
            jul.append(jul_weather)
            aug.append(aug_weather)
            sep.append(sep_weather)
            
            print("working")
        
    except:
        jul.append("error")
        aug.append("error")
        sep.append("error")
        
        print("not working")
else:
    try:
        lat = winery["latitude"]
        lng = winery["longitude"]

        jan_weather = requests.get(f"{base_url}{lat},{lng}&key={weather_api}&date=2011-01-01&enddate=2011-01-31&tp=24&format=json").json()
        feb_weather = requests.get(f"{base_url}{lat},{lng}&key={weather_api}&date=2011-02-01&enddate=2011-02-28&tp=24&format=json").json()
        mar_weather = requests.get(f"{base_url}{lat},{lng}&key={weather_api}&date=2011-03-01&enddate=2011-03-31&tp=24&format=json").json()
                
        jan.append(jul_weather)
        feb.append(aug_weather)
        mar.append(sep_weather)
        
        print("working")
    except:
        jan.append("error")
        feb.append("error")
        mar.append("error")   
        
        print("not working")
    
        